In [ ]:
import cv2
import os
import shutil

def apply_gaussian_blur(img, kernel_size=(3, 3), sigma=0.5):
    return cv2.GaussianBlur(img, kernel_size, sigma)

def apply_bilateral_filter(img, d=9, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(img, d, sigma_color, sigma_space)

def adjust_brightness_contrast(img, brightness=20, contrast=20):
    if brightness != 0:
        shadow = brightness if brightness > 0 else 0
        highlight = 255 if brightness > 0 else 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow
        img = cv2.addWeighted(img, alpha_b, img, 0, gamma_b)

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)
        img = cv2.addWeighted(img, alpha_c, img, 0, gamma_c)

    return img

def resize_image(img, size=(416, 416)):
    return cv2.resize(img, size)

def preprocess_image(img_path, output_path, size=(416, 416), brightness=30, contrast=30):
    img = cv2.imread(img_path)
    if img is not None:
        img = apply_gaussian_blur(img)
        img = apply_bilateral_filter(img)
        img = adjust_brightness_contrast(img, brightness, contrast)
        img = resize_image(img, size)
        cv2.imwrite(output_path, img)
    else:
        print(f"Error reading image: {img_path}")

def preprocess_dataset(input_image_dir, input_label_dir, output_image_dir, output_label_dir, size=(416, 416), brightness=30, contrast=30):
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)

    print(f"Processing images in: {input_image_dir}")
    print(f"Saving processed images to: {output_image_dir}")

    image_count = 0
    for filename in os.listdir(input_image_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_count += 1
            preprocess_image(os.path.join(input_image_dir, filename), os.path.join(output_image_dir, filename), size, brightness, contrast)
            # Sao chép nhãn
            label_file = filename.rsplit('.', 1)[0] + '.txt'
            shutil.copy(os.path.join(input_label_dir, label_file), os.path.join(output_label_dir, label_file))
    print(f"Total images processed: {image_count}")

# Chạy tiền xử lý dữ liệu
preprocess_dataset('/content/wb_localization_dataset/images/train', '/content/wb_localization_dataset/labels/train', '/content/preprocessed_data/train/images', '/content/preprocessed_data/train/labels')
preprocess_dataset('/content/wb_localization_dataset/images/val', '/content/wb_localization_dataset/labels/val', '/content/preprocessed_data/val/images', '/content/preprocessed_data/val/labels')